# 1. Загружаем библиотеки

In [1]:
import math
import numpy as np
import sklearn
import sklearn.model_selection

# 2. Функции вычисления энтропии

entropy_func-математическая функция.
entropy_cal-возвращает энтропию группы данных.с1-кол-во одного класса, с2-кол-во другого класса.
entropy_one_division-возвращает энтропию разделенной группы данных.
get_entropy-энтропия разделения.
y_predict - это разделенное решение, True / False и y_true может быть мульти-классом

In [2]:
def entropy_func(c, n):
    p = c * 1.0 / n
    return -p * math.log(p, 2)

def entropy_cal(c1, c2):
    if c1== 0 or c2 == 0: # если всего один класс в группе, то энтропия =0
        return 0    
    return entropy_func(c1, c1+c2) + entropy_func(c2, c1+c2)

def entropy_one_division(division):
    s = 0
    n = len(division)
    classes = set(division)
    for c in classes:# найти энтропию для каждого класса
        n_c = sum(division==c)
        s += n_c*1.0/n * entropy_cal(sum(division==c), sum(division!=c))# средне взвешенная
    return s, n

def get_entropy(y_predict, y_real):
    if len(y_predict) != len(y_real):
        print('y_predict и y должны быть одной длины')
        return None
    n = len(y_real)
    s_true, n_true = entropy_one_division(y_real[y_predict]) # левая часть энтропии
    s_false, n_false = entropy_one_division(y_real[~y_predict]) # правая часть энтропии
    s = n_true*1.0/n * s_true + n_false*1.0/n * s_false # общая средневзешенная энтропия
    return s

# 3. Класс DesisionTreeClass
В нем x- матрица входных параметров, y-вектор исходов, par_node- дерево, сгенерированное для x и y, 
depth- глубина текущего слоя.


In [3]:
class DecisionTreeClass(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
    
    def fit(self, x, y, par_node={}, depth=0):
        if par_node is None: #case 1-дерево останавливается на предыдущем уровне
            print(f"parnode is none. \n{par_node}")
            return None
        elif len(y) == 0: #case 2 нет данных
            return None
        elif self.all_same(y): # case 3 все y однинаковы
            return {'val':y[0]}
        elif depth >= self.max_depth: # case 4  достигнута макс длина
            return None
        else: 
        #находим split с учетом полученной информации
            col, cutoff, entropy = self.split_all(x, y)
            y_left = y[x[:, col] < cutoff] #данные слева
            y_right = y[x[:, col] >= cutoff] #данные справа
            par_node = {'col': iris.feature_names[col], 'index_col':col,
                        'cutoff':cutoff,
                       'val': np.round(np.mean(y))} # сохраняем информацию
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1) #генерация дерева для левых данных
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1) #генерация дерева для правых данных
            self.depth += 1 #увеличение глубины
            self.trees = par_node
            return par_node
    
    def split_all(self, x, y):
        col = None
        min_entropy = 10
        cutoff = None
        for i, c in enumerate(x.T): # итерация каждой функции
            entropy, cur_cutoff = self.split(c, y)# найти лучшее разделение этой функции
            if entropy == 0:# находим первую отсечку
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:# проверка
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        return col, cutoff, min_entropy
    
    def split(self, col, y):
        """
        Внутренняя функция. Разбивает матрицу на 2 смежные части, максимизируя критерий.
        """
        min_entropy = 10
        n = len(y)
        for value in set(col): # итерация по каждому значению в столбце
            y_predict = col < value # разделяем y на 2 группы
            m_entropy = get_entropy(y_predict, y)
            if m_entropy <= min_entropy:
                min_entropy = m_entropy
                cutoff = value
        return min_entropy, cutoff
    
    def all_same(self, items):
        return all(x == items[0] for x in items)
                                           
    def predict(self, x):
        """
        Выводит вектор предсказаний для заданной матрицы X
        """
        results = np.array([0]*len(x))
        for i, c in enumerate(x):# для каждой строки в тестовых данных
            results[i] = self._predict(c)
        return results
    def _predict(self, row):
        tree = self.trees# получаем дерево, которое мы строим в обучении
        while tree.get('cutoff'):# если не листовой узел
            if row[tree['index_col']] < tree['cutoff']:# получить направление
                if not tree['left']:
                    return tree.get('val')
                tree = tree['left']
                 
            else:
                if not tree['right']:# если листовой узел,то возвращаем значение
                    return tree.get('val')
                tree = tree['right']
        else:
            return tree.get('val')
    


# 4. Подготовка и тестирование данных.
Загружаем датасет wine с помощью sklearn.datasets

In [4]:
from sklearn.datasets import load_wine
from pprint import pprint
from sklearn.model_selection import train_test_split
iris = load_wine()
x = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,test_size=0.3, random_state=121)
clf = DecisionTreeClass(max_depth=20)
m = clf.fit(X_train, y_train)

pprint(m)

{'col': 'color_intensity',
 'cutoff': 3.7,
 'index_col': 9,
 'left': {'val': 1},
 'right': {'col': 'proline',
           'cutoff': 880.0,
           'index_col': 12,
           'left': {'col': 'flavanoids',
                    'cutoff': 1.41,
                    'index_col': 6,
                    'left': {'val': 2},
                    'right': {'col': 'magnesium',
                              'cutoff': 102.0,
                              'index_col': 4,
                              'left': {'val': 1},
                              'right': {'val': 0},
                              'val': 1.0},
                    'val': 2.0},
           'right': {'val': 0},
           'val': 1.0},
 'val': 1.0}


In [5]:
y_pred = clf.predict(X_test)
def accuracy(y_pred, y_test):
    return sum(y_pred == y_test) / len(y_pred)
accuracy(y_pred, y_test)

0.8888888888888888